# Data Aggregation and Group Operations

* Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow.
* After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot tables for reporting or visualization purposes.
* Pandas provides a flexible groupby interface, enabling you to slice, dice, and summarize datasets in a natural way.
* One reason for the popularity of relational databases and SQL  is the ease with which data can be joined, filtered, transformed, and aggregated. However, query languages like SQL are somewhat constrained in the kinds of group operations that can be performed.
* As you will see, with the expressiveness of Python and pandas, we can perform quite complex group operations by utilizing any function that accepts a pandas object or NumPy array.
* Objective of this chapter includes:
> * Split a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names).
> * Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function.
> * Apply within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection.
> * Compute pivot tables and cross-tabulations.
> * Perform quantile analysis and other statistical group analyses.



In [1]:
import numpy as np
import pandas as pd

## GroupBy Mechanics

* The term ***split-apply-combine*** is used for describing group operations.
* In the first stage of the process, data contained in a pandas object, whether a Series, Data Frame, or otherwise, is split into groups based on one or more keys that you provide.
* The splitting is performed on a particular axis of an object
> * Ex: A DataFrame can be grouped on its rows (axis=0) or its columns (axis=1).
* Once this is done, a function is applied to each group, producing a new value.
* Finally, the results of all those function applications are combined into a result object.
* The form of the resulting object will usually depend on what’s being done to the data.
* Figure below for a mockup of a simple group aggregation.

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/29_orig.png" width = "400 px" height = "300 px">
</p>


* Each grouping key can take many forms, and the keys do not have to be all of the same type:
> * A list or array of values that is the same length as the axis being grouped
> * A value indicating a column name in a DataFrame
> * A dict or Series giving a correspondence between the values on the axis being grouped and the group names
> * A function to be invoked on the axis index or the individual labels in the index
---
* Consider a small tabular dataset as a DataFrame:

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.207451,-0.956559
1,a,two,-1.072514,0.227722
2,b,one,-1.466418,1.468443
3,b,two,1.183073,-1.632346
4,a,one,0.439691,-1.736253


* Suppose you wanted to compute the mean of the data1 column using the labels from key1.
* There are a number of ways to do this. One is to access data1 and call groupby with the column (a Series) at key1:

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

* This grouped variable is now a ***GroupBy object***. It has not actually computed anything yet except for some intermediate data about the group key df['key1'].
* The idea is that this object has all of the information needed to then apply some operation to each of the groups.
* Ex: to compute group means we can call the GroupBy’s mean method:

In [4]:
grouped.mean()

key1
a   -0.141791
b   -0.141672
Name: data1, dtype: float64

* The important thing here is that the data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column.
* The result index has the name 'key1' because the DataFrame column df['key1'] did.
* If instead we had passed multiple arrays as a list, we’d get something different:

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.323571
      two    -1.072514
b     one    -1.466418
      two     1.183073
Name: data1, dtype: float64

* Here we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.323571,-1.072514
b,-1.466418,1.183073


* In this example, the group keys are all Series, though they could be any arrays of the right length:


In [7]:
city = np.array(['Manipal', 'Udupi', 'Udupi', 'Manipal', 'Manipal'])
city

array(['Manipal', 'Udupi', 'Udupi', 'Manipal', 'Manipal'], dtype='<U7')

In [8]:
years = np.array([2005, 2005, 2006, 2005, 2006])
years

array([2005, 2005, 2006, 2005, 2006])

In [9]:
df['data1'].groupby([city, years]).mean()

Manipal  2005    0.695262
         2006    0.439691
Udupi    2005   -1.072514
         2006   -1.466418
Name: data1, dtype: float64

* Frequently the grouping information is found in the same DataFrame as the data you want to work on.
* In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:

In [10]:
df.groupby('key1').mean()

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\2057825055.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('key1').mean()


,data1,data2
key1,,
a,-0.141791,-0.821697
b,-0.141672,-0.081952


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.323571 -1.346406
     two  -1.072514  0.227722
b    one  -1.466418  1.468443
     two   1.183073 -1.632346

* Noticed here in the first case df.groupby('key1').mean() that there is no key2 column in the result.
* Because df['key2'] is not numeric data, it is said to be a
nuisance column, which is therefore excluded from the result.
* By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset.
* Regardless of the objective in using groupby, a generally useful GroupBy method is size, which returns a Series containing group sizes:

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

* ***Note:*** any missing values in a group key will be excluded from the result.

### Iterating Over Groups

* The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.
* Consider the following:

In [13]:
for name, group in df.groupby('key1'):
    print(name)
    #print(group)

a
b


* In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [14]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.207451 -0.956559
4    a  one  0.439691 -1.736253
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.072514  0.227722
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.466418  1.468443
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.183073 -1.632346


* Of course, you can choose to do whatever you want with the pieces of data.
* A recipe you may find useful is computing a dict of the data pieces as a one-liner:

In [15]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.466418,1.468443
3,b,two,1.183073,-1.632346


In [16]:
pieces['a']

,key1,key2,data1,data2
0,a,one,0.207451,-0.956559
1,a,two,-1.072514,0.227722
4,a,one,0.439691,-1.736253


* By default groupby groups on axis=0, but you can group on any of the other axes.
* Ex: we could group the columns of our example df here by dtype like so:

In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped = df.groupby(df.dtypes, axis=1)
grouped

* We can print out the groups like so:

In [19]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.207451 -0.956559
1 -1.072514  0.227722
2 -1.466418  1.468443
3  1.183073 -1.632346
4  0.439691 -1.736253
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

* Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.
* This means that:
> `df.groupby('key1')['data1']`
> `df.groupby('key1')[['data2']]`

* are syntactic sugar for:
> `df['data1'].groupby(df['key1'])`
> `df[['data2']].groupby(df['key1'])`

* Especially for large datasets, it may be desirable to aggregate only a few columns.
* Ex: In the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:

In [20]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.346406
     two   0.227722
b    one   1.468443
     two  -1.632346

* The object returned by this indexing operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single column name is passed as a scalar:

In [21]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [22]:
s_grouped.mean()

key1  key2
a     one    -1.346406
      two     0.227722
b     one     1.468443
      two    -1.632346
Name: data2, dtype: float64

### Grouping with Dicts and Series

* Grouping information may exist in a form other than an array.
* Let’s consider another example DataFrame:

In [23]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.114867,0.704674,0.328995,0.930256,0.884291
Steve,-0.371809,0.584045,-1.352386,0.854419,-0.339446
Wes,-0.314880,NaN,NaN,1.265140,0.143981
Jim,-0.174054,-0.126783,1.342884,0.773665,-0.418066
Travis,-0.367424,-1.267610,-0.170432,0.956386,-0.885143


Now, let's break down the code people.iloc[2:3, [1, 2]] = np.nan:
> * people.iloc[2:3, [1, 2]]: This part of the code uses the iloc indexer to select a specific subset of rows and columns from the DataFrame.
> * 2:3 selects rows from index 2 (inclusive) to index 3 (exclusive), so it selects only the row with index 2 (which is the third row).
> * [1, 2] selects columns with index 1 and 2.
> * = np.nan: This part assigns NaN values to the selected subset of the DataFrame.

* Now, suppose we have a group correspondence for the columns and want to sum together the columns by group:

In [24]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

* Now, you could construct an array from this dict to pass to groupby, but instead we can just pass the dict (I included the key 'f' to highlight that unused grouping keys
are OK):

In [25]:
by_column = people.groupby(mapping, axis=1)
by_column

In [26]:
by_column.sum()

,blue,red
Joe,1.259251,1.703832
Steve,-0.497966,-0.127210
Wes,1.265140,-0.170899
Jim,2.116549,-0.718903
Travis,0.785954,-2.520178


* The same functionality holds for Series, which can be viewed as a fixed-size mapping:

In [27]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [28]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

* Using Python functions is a more generic way of defining a group mapping compared with a dict or Series.
* Any function passed as a group key will be called once per index value, with the return values being used as the group names. More concretely, consider the example DataFrame from the previous section, which has people’s first names as index values.
* Suppose you wanted to group by the length of the names; while you could compute an array of string lengths, it’s simpler to just pass the len function:

In [29]:
people

,a,b,c,d,e
Joe,0.114867,0.704674,0.328995,0.930256,0.884291
Steve,-0.371809,0.584045,-1.352386,0.854419,-0.339446
Wes,-0.314880,NaN,NaN,1.265140,0.143981
Jim,-0.174054,-0.126783,1.342884,0.773665,-0.418066
Travis,-0.367424,-1.267610,-0.170432,0.956386,-0.885143


Now, let's break down the code people.groupby(len).sum():

> * people.groupby(len): This part groups the DataFrame people by the length of the index labels (i.e., the length of the 'Name' column values). In this example, it groups the data based on the number of characters in the names.

> * .sum(): After grouping, this part calculates the sum of numerical columns for each group. It sums the individuals whose names have the same length.

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.374067,0.577892,1.671879,2.969061,0.610205
5,-0.371809,0.584045,-1.352386,0.854419,-0.339446
6,-0.367424,-1.267610,-0.170432,0.956386,-0.885143


* Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
key_list

['one', 'one', 'one', 'two', 'two']

In [32]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.314880  0.704674  0.328995  0.930256  0.143981
  two -0.174054 -0.126783  1.342884  0.773665 -0.418066
5 one -0.371809  0.584045 -1.352386  0.854419 -0.339446
6 two -0.367424 -1.267610 -0.170432  0.956386 -0.885143

### Grouping by Index Levels

* A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index.
* Let’s look at an example:

In [33]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns


MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [34]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.754619 -0.762542  0.403443  0.009902  0.635189
1     -0.365698  0.378705  0.775286  1.215549  1.541485
2     -0.759625  1.071435 -0.494576 -0.793243 -0.979714
3     -1.356132  1.205437  1.996718  1.731663  0.106855

* To group by level, pass the level number or name using the level keyword:

In [35]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


`hier_df.groupby(level='cty', axis=1).count()`: This line groups the columns of the DataFrame hier_df by the 'cty' level along the columns (axis=1) and then calculates the count of non-null values within each group. In other words, it counts how many non-null values are in each 'cty' group.

## Data Aggregation

* Aggregations refer to any data transformation that produces scalar values from arrays.
* The preceding examples have used several of them, including mean, count, min, and sum.
* You may wonder what is going on when you invoke mean() on a
GroupBy object. Many common aggregations, such as those found in Table below have optimized implementations.
* However, you are not limited to only this set of methods.
---
<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/30_orig.png" width = "600 px" height = "300 px">
</p>

---

* You can use aggregations of your own devising and additionally call any method that is also defined on the grouped object.
* Ex: you might recall that quantile computes sample quantiles of a Series or a DataFrame’s columns.
* While quantile is not explicitly implemented for GroupBy, it is a Series method and thus available for use.
* Internally, GroupBy efficiently slices up the Series, calls piece.quantile(0.9) for each piece, and then assembles those results together into the result object:

In [36]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.151466,-1.746352
1,a,two,-0.523690,-0.142583
2,b,one,0.095177,-0.128236
3,b,two,-0.894384,0.297014
4,a,one,-0.399352,1.000458


In [37]:
grouped = df.groupby('key1')
grouped
grouped['data1'].quantile(0.9)

key1
a    0.041303
b   -0.003779
Name: data1, dtype: float64

* key1 and key2 are categorical columns.
* data1 and data2 contain random numerical data.

Now, let's explain the code step by step:

* grouped = df.groupby('key1'): This line groups the DataFrame df by the 'key1' column, creating two groups ('a' and 'b') based on the unique values in the 'key1' column.

* grouped['data1'].quantile(0.9): This line calculates the 90th percentile (0.9 quantile) of the 'data1' column within each group.
> * For the 'a' group, it calculates the 90th percentile of 'data1' for rows where 'key1' is 'a'.
> * For the 'b' group, it calculates the 90th percentile of 'data1' for rows where 'key1' is 'b'.

* To use your own aggregation functions, pass any function that aggregates an array to the aggregate or agg method:

In [38]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\663192242.py:3: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,0.675156,2.746810
b,0.989561,0.425249


* The above code defines a custom Python function called peak_to_peak. This function takes an array arr as input and calculates the peak-to-peak range by subtracting the minimum value from the maximum value.

`grouped.agg(peak_to_peak)`

* The above line applies the peak_to_peak function to each group within the grouped object. Specifically, it calculates the peak-to-peak range of the numerical columns ('data1' and 'data2') for each group.
---

* You may notice that some methods like describe also work, even though they are not aggregations, strictly speaking:

In [39]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.257192  0.359327 -0.523690 -0.461521 -0.399352 -0.123943   
b      2.0 -0.399604  0.699725 -0.894384 -0.646994 -0.399604 -0.152213   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.151466   3.0 -0.296159  1.379830 -1.746352 -0.944467 -0.142583   
b     0.095177   2.0  0.084389  0.300697 -0.128236 -0.021923  0.084389   

                          
           75%       max  
key1                      
a     0.428938  1.000458  
b     0.190702  0.297014

### Column-Wise and Multiple Function Application

* Consider the tipping dataset. After loading it with read_csv, we add a tipping percentage column tip_pct:

In [40]:
tips = pd.read_csv('tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


* As you’ve already seen, aggregating a Series or all of the columns of a DataFrame is a matter of using aggregate with the desired function or calling a method like mean or std.
* ***However, you may want to aggregate using a different function depending on the column, or multiple functions at once.***
* First, I’ll group the tips by day and smoker:

In [41]:
grouped = tips.groupby(['day', 'smoker'])
grouped

Note that for descriptive statistics, you can pass the name of the function as a string:

In [42]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

* If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions:

In [43]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

* Here we passed a list of aggregation functions to agg to evaluate indepedently on the data groups.
---
* You don’t need to accept the names that GroupBy gives to the columns; notably, lambda functions have the name "lambda", which makes them hard to identify.
* Thus, if you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names (you can think of a list of 2-tuples as an ordered mapping):

In [44]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

* With a DataFrame you have more options, as you can specify a list of functions to
apply to all of the columns or different functions per column.
* To start, suppose we wanted to compute the same three statistics for the tip_pct and total_bill columns:

In [45]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\576261660.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

* As you can see, the resulting DataFrame has hierarchical columns, the same as you would get aggregating each column separately and using concat to glue the results together using the column names as the keys argument:

In [46]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [47]:
result['total_bill']

count       mean    max
day  smoker                         
Fri  No          4  18.420000  22.75
     Yes        15  16.813333  40.17
Sat  No         45  19.661778  48.33
     Yes        42  21.276667  50.81
Sun  No         57  20.506667  48.17
     Yes        19  24.120000  45.35
Thur No         45  17.113111  41.19
     Yes        17  19.190588  43.11

* As before, a list of tuples with custom names can be passed:

In [48]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\3909980601.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

* Now, ***suppose you wanted to apply potentially different functions to one or more of the columns***.
* To do this, pass a dict to agg that contains a mapping of column names to any of the function specifications listed so far:

In [49]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [50]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

* A DataFrame will have hierarchical columns only if multiple functions are applied to at least one column.


### Returning Aggregated Data Without Row Indexes

* In all of the examples up until now, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations.
* Since this isn’t always desirable, you can disable this behavior in most cases by passing as_index=False to groupby:

In [51]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\2496195394.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(['day', 'smoker'], as_index=False).mean()


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


* Of course, it’s always possible to obtain the result in this format by calling reset_index on the result. Using the as_index=False method avoids some unnecessary computations.

## Apply: General split-apply-combine

* The most general-purpose GroupBy method is apply, as illustrated in Figure below, apply splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together.
---
<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/31_orig.png" width = "400 px" height = "300 px">
</p>

---
* Returning to the tipping dataset from before, suppose you wanted to select the top five tip_pct values by group.
* First, write a function that selects the rows with the largest values in a particular column:

In [52]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


* top is a custom function that takes three arguments:

  > *  `df: The DataFrame to be sorted and filtered.`
  > *  `n: The number of rows to return (default is 5).`
  > * `column: The column by which to sort the DataFrame (default is 'tip_pct')`

* Inside the function, it sorts the DataFrame by the specified column in descending order and returns the top n rows.
---

* Now, if we group by smoker, say, and call apply with this function, we get the following:

In [78]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

* This line groups the tips DataFrame by the 'smoker' column and then applies the top function to each group.
* It sorts each group by the 'tip_pct' column (the sorting column specified in the top function) and returns the top rows from each group.
* The result will be a DataFrame that shows the top rows for each group (smoker = 'Yes' and smoker = 'No') based on the 'tip' column.
---

* If you pass a function to apply that takes other arguments or keywords, you can pass these after the function:

In [79]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

* The above code is using the groupby() method on the tips DataFrame to group the data by two columns: 'smoker' and 'day'.
* Then, it is applying the custom top function to each group within these combinations of 'smoker' and 'day' with specific arguments (n=1, column='total_bill').


* ***tips.groupby(['smoker', 'day']):*** This part groups the tips DataFrame by two columns, 'smoker' and 'day', creating a multi-level index.

* ***.apply(top, n=1, column='total_bill'):*** For each group created by the combination of 'smoker' and 'day', it applies the custom top function. The top function is applied with specific arguments:
> * n=1: It specifies to return only the top 1 row for each group.
> * column='total_bill': It specifies to sort the groups based on the 'total_bill' column.

* You may recall that I earlier called describe on a GroupBy object:

In [55]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [56]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

* Inside GroupBy, when you invoke a method like describe, it is actually just a shortcut for:

> *  `f = lambda x: x.describe()`
> *  `grouped.apply(f)`

### Suppressing the Group Keys

* In the preceding examples, you see that the resulting object has a hierarchical index formed from the group keys along with the indexes of each piece of the original object.
* You can disable this by passing group_keys=False to groupby:

In [57]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

* If you recall, pandas has some tools, in particular cut and qcut, for slicing data up into buckets with bins of your choosing or by sample quantiles.
* Combining these functions with groupby makes it convenient to perform bucket or quantile analysis on a dataset. Consider a simple random dataset and an equal-length bucket categorization using cut:


In [58]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

frame

,data1,data2
0,0.432523,0.093533
1,-0.188765,-1.208122
2,-1.524445,-0.060346
3,1.168409,0.398974
4,-0.108308,-0.177281
...,...,...
995,-0.554275,-1.377828
996,0.173138,0.692698
997,0.586123,-0.195636
998,0.542961,0.081092


In [59]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-0.405, 1.218]
1     (-0.405, 1.218]
2    (-2.027, -0.405]
3     (-0.405, 1.218]
4     (-0.405, 1.218]
5     (-0.405, 1.218]
6    (-2.027, -0.405]
7    (-2.027, -0.405]
8      (1.218, 2.841]
9     (-0.405, 1.218]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.656, -2.027] < (-2.027, -0.405] < (-0.405, 1.218] < (1.218, 2.841]]

* In the above code, the ***pd.cut() function*** is applied to the ***'data1'*** column of the frame DataFrame.
* ***The pd.cut() function is used to bin the values in 'data1' into intervals (quartiles, in this case)***.
* The 4 as the second argument specifies that we want to divide the values into 4 equal-width bins (quartiles).
* The resulting quartiles variable contains categorical data representing which quartile each 'data1' value falls into.
* Next line displays the first 10 rows of the quartiles Series, which shows how the first 10 'data1' values have been categorized into quartiles. Each value in the quartiles Series represents the quartile range to which the corresponding 'data1' value belongs.
---

* The Categorical object returned by cut can be passed directly to groupby.
* So we could compute a set of statistics for the data2 column like so:

In [60]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.656, -2.027]",-2.196622,1.991485,28.0,-0.123168
"(-2.027, -0.405]",-2.821810,2.724978,343.0,-0.010447
"(-0.405, 1.218]",-3.267451,3.114560,524.0,0.054649
"(1.218, 2.841]",-2.367679,2.503412,105.0,0.152485


* These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use qcut.
* Lets pass labels=False to just get quantile numbers:

In [61]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.196622,2.381566,100.0,0.002319
1,-2.525050,2.055501,100.0,-0.008435
2,-2.821810,2.602133,100.0,-0.023852
3,-2.762742,2.724978,100.0,-0.053633
4,-2.865809,3.114560,100.0,0.052435
5,-2.988253,2.068712,100.0,0.047367
6,-2.562539,2.458788,100.0,-0.050815
7,-2.876347,3.069800,100.0,0.288665
8,-3.267451,2.007299,100.0,-0.019928


### Example: Filling Missing Values with Group-Specific       Values

* When cleaning up missing data, in some cases you will replace data observations using dropna, but in others you may want to impute (fill in) the null (NA) values using a fixed value or some value derived from the data.
* fillna is the right tool to use;
* Ex: here we fill in NA values with the mean:

In [62]:
s = pd.Series(np.random.randn(6))
s

0    0.126373
1   -0.657720
2    0.054886
3   -0.263045
4   -0.026283
5   -0.967948
dtype: float64

In [63]:
s[::2] = np.nan
s

0         NaN
1   -0.657720
2         NaN
3   -0.263045
4         NaN
5   -0.967948
dtype: float64

* In the above code, we use slicing ([::2]) to select every other element in the Series and set them to np.nan, which represents a missing value in Pandas. As a result, every other value in the Series is replaced with NaN.

In [64]:
s.fillna(s.mean())

0   -0.629571
1   -0.657720
2   -0.629571
3   -0.263045
4   -0.629571
5   -0.967948
dtype: float64

* The above code fills the missing values in the Series s with the mean of the non-missing values in the Series. Here's how it works:
> * s.mean(): Calculates the mean of the non-missing values in the Series.
> * s.fillna(s.mean()): Replaces the NaN values in the Series with the calculated mean.

* However, it's important to note that this line of code computes the mean of the Series before filling the NaN values. The result will be a new Series with missing values filled in with the mean. The original Series s is not modified in place.
* If you want to modify s in place, you can use s.fillna(s.mean(), inplace=True).
---

* ***Suppose you need the fill value to vary by grou***p.
* One way to do this is to group the data and use apply with a function that calls fillna on each data chunk.
* Here is some sample data on US states divided into eastern and western regions:

In [65]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.152650
New York     -0.106807
Vermont      -0.409943
Florida       1.591200
Oregon       -0.712830
Nevada       -1.345942
California   -1.205759
Idaho        -0.301289
dtype: float64

* The above code creates a Pandas Series called data using NumPy-generated random data and assigns custom index labels to it.
* It then sets up a list called group_key to categorize the data into two groups, 'East' and 'West,' and assigns these group labels to the data using the index.
* It assigns the label 'East' to the first four states ('Ohio', 'New York', 'Vermont', 'Florida') and the label 'West' to the next four states ('Oregon', 'Nevada', 'California', 'Idaho'). This is done using the multiplication operator (*) to repeat the labels.
---

* Note that the syntax ['East'] * 4 produces a list containing four copies of the elements in ['East']. Adding lists together concatenates them.
* Let’s set some values in the data to be missing:

In [66]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.152650
New York     -0.106807
Vermont            NaN
Florida       1.591200
Oregon       -0.712830
Nevada             NaN
California   -1.205759
Idaho              NaN
dtype: float64

In [67]:
data.groupby(group_key).mean()

East    0.545681
West   -0.959294
dtype: float64

* * We can fill the NA values using the group means like so:

In [68]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\649295739.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio          0.152650
New York     -0.106807
Vermont       0.545681
Florida       1.591200
Oregon       -0.712830
Nevada       -0.959294
California   -1.205759
Idaho        -0.959294
dtype: float64

* In another case, you might have predefined fill values in your code that vary by group.
* Since the groups have a name attribute set internally, we can use that:

In [69]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\2621590873.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_func)


Ohio          0.152650
New York     -0.106807
Vermont       0.500000
Florida       1.591200
Oregon       -0.712830
Nevada       -1.000000
California   -1.205759
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

* Suppose you wanted to draw a random sample (with or without replacement) from a large dataset for simulation purposes or some other application.
* There are a number of ways to perform the “draws”; here we use the sample method for Series.
* To demonstrate, here’s a way to construct a deck of English-style playing cards:

In [70]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

* So now we have a Series of length 52 whose index contains card names and values are the ones used in Blackjack and other games (to keep things simple, I just let the ace 'A' be 1):

In [71]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

* Now, based on what I said before, drawing a hand of five cards from the deck could be written as:

In [72]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

5S      5
KC     10
JD     10
10S    10
8D      8
dtype: int64

* Suppose you wanted two random cards from each suit. Because the suit is the last character of each card name, we can group based on this and use apply:

In [73]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  3C     3
   QC    10
D  4D     4
   8D     8
H  7H     7
   QH    10
S  4S     4
   2S     2
dtype: int64

* Alternatively, we could write:

In [74]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

3C    3
4C    4
2D    2
4D    4
5H    5
3H    3
5S    5
8S    8
dtype: int64

### Example: Group Weighted Average and Correlation

* Under the split-apply-combine paradigm of groupby, operations between columns in a DataFrame or two Series, such as a group weighted average, are possible.
* Ex: take this dataset containing group keys, values, and some weights:

In [80]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.697288,0.598454
1,a,0.061922,0.897544
2,a,0.054874,0.898899
3,a,1.218737,0.035259
4,b,-0.691069,0.731415
5,b,-0.894356,0.531320
6,b,0.450029,0.163510
7,b,0.639387,0.801747


* The group weighted average by category would then be:

In [81]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.232566
b   -0.177037
dtype: float64

* Ex2: consider a financial dataset originally obtained from Yahoo! Finance containing end-of-day prices for a few stocks and the S&P 500 index (the SPX symbol):

In [82]:
close_px = pd.read_csv('stock_px.csv', parse_dates=True,
                       index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [83]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


* One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with SPX.
* As one way to do this, we first create a function that computes the pairwise correlation of each column with the 'SPX' column:

In [84]:
spx_corr = lambda x: x.corrwith(x['SPX'])

* Next, we compute percent change on close_px using pct_change:

In [85]:
rets = close_px.pct_change().dropna()

* Lastly, we group these percent changes by year, which can be extracted from each row label with a one-line function that returns the year attribute of each datetime label:

In [86]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


* You could also compute inter-column correlations. Here we compute the annual correlation between Apple and Microsoft:

In [87]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

* In the same theme as the previous example, you can use groupby to perform more complex group-wise statistical analysis, as long as the function returns a pandas object or scalar value.
* Ex: I can define the following regress function (using the statsmodels econometrics library), which executes an ordinary least squares (OLS) regression on each chunk of data:

In [88]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

* Now, to run a yearly linear regression of AAPL on SPX returns, execute:

In [89]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation

* ***A pivot table is a data summarization tool*** frequently found in spreadsheet programs and other data analysis software.
* It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.
* Pivot tables in Python with pandas are made possible through the groupby facility combined with reshape operations utilizing hierarchical indexing.
* DataFrame has a pivot_table method, and there is also a top-level pandas.pivot_table function.
* In addition to providing a convenience interface to groupby, pivot_table can add partial totals, also known as margins.
* Returning to the tipping dataset, suppose you wanted to compute a table of group means (the default pivot_table aggregation type) arranged by day and smoker on the rows:

In [90]:
tips.pivot_table(index=['day', 'smoker'])

C:\Users\appur\AppData\Local\Temp\ipykernel_15720\2966571272.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  tips.pivot_table(index=['day', 'smoker'])


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

* This could have been produced with groupby directly. Now, suppose we want to aggregate only tip_pct and size, and additionally group by time.
* IF we put smoker in the table columns and day in the rows:

In [91]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

* We could augment this table to include partial totals by passing margins=True.
* This has the effect of adding All row and column labels, with corresponding values being the group statistics for all the data within a single tier:

In [92]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

* Here, the All values are means without taking into account smoker versus nonsmoker (the All columns) or any of the two levels of grouping on the rows (the All row).
---
* To use a different aggregation function, pass it to aggfunc.
* Ex: 'count' or len will give you a cross-tabulation (count or frequency) of group sizes:

In [93]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

* If some combinations are empty (or otherwise NA), you may wish to pass a fill_value:

In [99]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

* Table below a summary of pivot_table methods

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/32_orig.png" width = "800 px" height = "300 px">
</p>



### Cross-Tabulations: Crosstab

* A cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies.
* Ex:

In [95]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [96]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


* As part of some survey analysis, we might want to summarize this data by nationality and handedness.
* You could use pivot_table to do this, but the pandas.crosstab
function can be more convenient:

In [97]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


* The first two arguments to crosstab can each either be an array or Series or a list of arrays. As in the tips data:

In [98]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244